## EM Algorithm

EM (Expectation Maximization) algorithm is a maximum likelihood estimation method for the situation with latent variables probabilistic model parameters. 

### Maximum Likelihood Estimation

MLE is a statistical parameter estimation method that is used for a random sample that satisfies a certain probability distribution, but the statistical parameters in it are unknown.

For example, there is a research to find out about the height distribution of students in a school. Firstly, assuming that the height of the students in the school follows a normal distribution $N(\mu, \sigma^{2})$, the parameters $\mu$ and $\sigma^{2}$ are unknown. And it's not realistic that testing all students' height, so the statistical sampling is used to do the task.

Then randomly selecting 100 students and using these data to estimate the height of all students. Because the selection of each person is independent, the probability of selecting these 100 people can be expressed as the product of the individual probabilities:
$$
L(\theta)=L\left(x_{1}, x_{2}, \ldots, x_{n} ; \theta\right)=\prod_{i=1}^{n} p\left(x_{i} \mid \theta\right)
$$

The above formula is the likelihood function. Usually for the convenience of calculation, it can be switched into the logarithm form:
$$
H(\theta)=\ln L(\theta)=\ln \prod_{i=1}^{n} p\left(x_{i} \mid \theta\right)=\sum_{i=1}^{n} \ln p\left(x_{i} \mid \theta\right)
$$

But among so many people in the school, why just these 100 people are selected? According to the theory of maximum likelihood estimation, it is precisely because the probability of these 100 people appearing is extremely high, that is, the maximum of its corresponding likelihood function:
$$
\hat{\theta} = \arg \max L(\theta)
$$

Find the partial derivative of $L(\theta)$ with respect to its parameter and set it to 0 can find the solution for the formula above. Therefore, the maximum likelihood estimation can be regarded as the inversion of the conditions from the sampling results, that is, it is known that a certain parameter can make the large probability of these samples appear, and the parameter can be used directly as the true value of the parameter estimation.

### Expectation Maximization

In the above example, it is unrealistic to assume that the height of both male and female students will follow the same distribution. When a sample is obtained, it's hard to tell that it's from male or female, so the height is the observable variable and the gender becomes the hidden variable in this situation. Therefore, there are two estimation problems in this case, and the normal MLE can't work well under a such circumstance. Now EM algorithm is introduced to solve this parameter estimation problems for probabilistic models including hidden variables.

If the observable variable data $Y$, hidden variable data $Z$, joint probability distribution $P(Y,Z|\theta)$ and conditional distribution of hidden variable $P(Z|Y,\theta)$ are given, the process of estimating the model parameter $\theta$ using the EM algorithm is as follows:

(1) Initialize the model parameter $\theta ^{(0)}$ and begin the iteration.

(2) Step E: mark $\theta ^{(i)}$ as the estimation for parameter in the $i$-th iteration, calculate Q function in the E step of the $i$+1-th iteration
$$
Q\left(\theta, \theta^{(i)}\right)=E_{Z}\left[\log P(Y, Z \mid \theta) \mid Y, \theta^{(i)}\right]=\sum_{Z} \log P(Y, Z \mid \theta) P\left(Z \mid Y, \theta^{(i)}\right)
$$

(3) Step M: Find the parameter that can maximize Q function and set it as the parameter estimation $\theta ^{(i+1)}$ in the $i$+1-th iteration
$$
\theta ^{(i+1)} = \arg \max_{\theta} Q(\theta, \theta ^{(i)})
$$

(4) Repeat iterative E step and M step until the convergence is satisfied.

From the EM algorithm process, it's clear that the key of the algorithm is to determine the Q function, step E estimates the distribution of the hidden variable with fixed model parameters, and step M estimates model parameters with the fixed hidden variable. The two are carried out interactively until the algorithm convergence conditions are met.

### Three-Coin Model

A classic example of the EM algorithm is the three-coin model. Suppose there are three coins $A$, $B$, and $C$, and the probabilities of appearing head of these three coins are $\pi$, $p$, and $q$, respectively. Use three coins to conduct the following experiment: first toss coin $A$, choose coin $B$ or $C$ according to the result, assuming that $B$ is selected for head and $C$ is selected for tail, and then the coin result is recorded, and the head is recorded as 1, and the tail is recorded as 0. Repeat the experiment 5 times independently. Each trial repeats tossing $B$ or $C$ 10 times. How to estimate the probability that each of the three coins will come up heads.

Since only the result of the final toss can be observed, there is no way to know whether this result was tossed by coin $A$ or by coin $B$. Therefore, in this process, which coin toss is selected according to the probability is a hidden variable. So we the EM algorithm can be used to solve this problem.

Step E: Initialize the probability that $B$ and $C$ appear heads to be $\hat{\theta}^{(0)}_{B}=0.6$ and $\hat{\theta}^{(0)}_{C}=0.5$, estimate the probability of selecting $B$ or $C$ in each experiment (the same as the probability $A$ appears head or tail). For example the probability of selecting $B$ is:
$$
P\left(z=B \mid y_{1}, \theta\right)=\frac{P\left(z=B, y_{1} \mid \theta\right)}{P\left(z=B, y_{1} \mid \theta\right)+P\left(z=C, y_{1} \mid \theta\right)}=\frac{(0.6)^{5} \times(0.4)^{5}}{(0.6)^{5} \times(0.4)^{5}+(0.5)^{10}}=0.45
$$

Correspondingly, the probability of selecting $C$ is $1$-$0.45$=$0.55$. The probability of choosing B and C for each trial is calculated and then weighted and summed based on the trial data.

Step M: Update the estimation of model parameter, and determine the value of parameter based on the Q function:
$$
\begin{aligned}
Q\left(\theta, \theta^{i}\right)&=\sum_{j=1}^{5} \sum_{z} P\left(z \mid y_{j}, \theta^{i}\right) \log P\left(z \mid y_{j}, \theta\right) \\
&=\sum_{j=1}^{5} \mu_{j} \log \left(\theta_{B}^{y_{j}}\left(1-\theta_{B}\right)^{10-y_{j}}\right)+\left(1-\mu_{j}\right) \log \left(\theta_{B}^{y_{j}}\left(1-\theta_{B}\right)^{1} 0-y_{j}\right)
\end{aligned}
$$

Differentiate the above formula and set it to 0 to get the parameter values after the first iteration: $\theta ^{(1)}_{B}=0.71, \enspace \theta ^{(1)}_{C}=0.58$. And then repeat the second, third ... iteration until the model converges.

The following is an implementation of a simple EM algorithm that procedure to solve the three-coin problem

In [1]:
import numpy as np

def em(data, thetas, max_iter=30, eps=1e-3):
    # initialize the likelihood function value
    ll_old = -np.infty
    for i in range(max_iter):
        # step E
        # log likelihood function
        log_like = np.array([np.sum(data * np.log(theta), axis=1) for theta in thetas])
        # likelihood function
        like = np.exp(log_like)
        # find the distribution of hidden variables
        ws = like/like.sum(0)
        # probability weighting
        vs = np.array([w[:, None] * data for w in ws])
        # step M: update parameter
        thetas = np.array([v.sum(0)/v.sum() for v in vs])
        # update log likelihood function
        ll_new = np.sum([w*l for w, l in zip(ws, log_like)])
        print("Iteration: %d" % (i+1))
        print("theta_B = %.2f, theta_C = %.2f, ll = %.2f" 
            % (thetas[0,0], thetas[1,0], ll_new))
        # terminate the iteration when the conditions are satisfied
        if np.abs(ll_new - ll_old) < eps:
            break
        ll_old = ll_new
    return thetas

In [2]:
# 5 independent trials with 10 flips per trial
# there are 5 heads and 5 tails in the first trial 
observed_data = np.array([(5,5), (9,1), (8,2), (4,6), (7,3)])
# initialize parameter, head for B is 0.6, and head for C is 0.5
thetas = np.array([[0.6, 0.4], [0.5, 0.5]])
thetas = em(observed_data, thetas, max_iter=30, eps=1e-3)

Iteration: 1
theta_B = 0.71, theta_C = 0.58, ll = -32.69
Iteration: 2
theta_B = 0.75, theta_C = 0.57, ll = -31.26
Iteration: 3
theta_B = 0.77, theta_C = 0.55, ll = -30.76
Iteration: 4
theta_B = 0.78, theta_C = 0.53, ll = -30.33
Iteration: 5
theta_B = 0.79, theta_C = 0.53, ll = -30.07
Iteration: 6
theta_B = 0.79, theta_C = 0.52, ll = -29.95
Iteration: 7
theta_B = 0.80, theta_C = 0.52, ll = -29.90
Iteration: 8
theta_B = 0.80, theta_C = 0.52, ll = -29.88
Iteration: 9
theta_B = 0.80, theta_C = 0.52, ll = -29.87
Iteration: 10
theta_B = 0.80, theta_C = 0.52, ll = -29.87
Iteration: 11
theta_B = 0.80, theta_C = 0.52, ll = -29.87
Iteration: 12
theta_B = 0.80, theta_C = 0.52, ll = -29.87
